In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
import numpy as np
import copy
from pprint import pprint

## 1. Cramer's rule

求解时间复杂度 $O(n^2)$，求解步骤:

1. 求解条件: 系数矩阵为方阵;
2. 求系数矩阵行列式:
    $$
    D = \left|
        \begin{array}\\
            a_{11} & a_{12} & ... & a_{1n}\\
            a_{21} & a_{22} & ... & a_{2n}\\
            ...    & ...    & ... & ...\\
            a_{n1} & a_{n2} & ... & a_{nn}\\
        \end{array}
    \right|
    $$
  系数行列式 $D = 0$ 有无穷解，不再求解;
3. 若 $D\neq 0$ 则方程有唯一解，各自变量的解为:
    $$
    x_j = \frac{D_{j}}{D}
    $$
    其中:
    $$
    D_{j}= \left|
        \begin{array} \\
            a_{11} & \cdots & a_{1, j-1} & b_{1}  & \cdots & a_{1 n} \\
            \vdots &        & \vdots     & \vdots &        & \vdots \\
            a_{n1} & \cdots & a_{n, j-1} & b_{1}  & \cdots & a_{n n}
        \end{array}
    \right|
    $$
4. 关于上面步骤的行列式:
    行列式: 某一行或一列的代数余子式之和，表示为:
    $$
    D=a_{i 1} A_{i 1}+a_{i 2} A_{i 2}+\ldots+a_{i n} A_{i n} \quad(i=1,2, \ldots, n)
    $$
    或者
    $$
    D=a_{1 j} A_{1 j}+a_{2 j} A_{2 j}+\ldots+a_{n j} A_{n j} \quad(j=1,2, \ldots, n)
    $$
    其中 代数余子式 $A_{ij}$:
    $$
    A_{i j}=(-1)^{i+j} M_{i j}
    $$
    余子式 $M_{i j}$: 划去 aij 所在的行和列，剩余的保留顺序的行列式。

In [52]:
def det(array):
    # print('\narray: \n', array)
    if len(array) == 1:
        return array[0][0]

    det_value = 0
    for i in range(len(array)):
        # print('array[i][0]: ', array[i][0])
        M = []  # 余子式 Mij，划去 aij 所在的第 i 行与第 j 列的元素
        for j in range(len(array)):
            if j != i:
                M.append(array[j][1:])
        # print('array[j][1:]: ', array[j][1:])
        det_value += (-1) ** i * array[i][0] * det(M)  # 代数余子式 Aij

    return det_value


def cramer(coeffi_array, beta):
    # 判断输入维度
    if len(coeffi_array) != len(beta):
        raise ValueError('系数矩阵与常数项维度不同！')
    if len(coeffi_array) != len(coeffi_array[0]):
        raise ValueError('系数矩阵不是方阵！')

    det_value = det(coeffi_array)  # 系数矩阵的行列式值
    if det_value == 0:
        raise ValueError('系数方阵行列式为 0，即存在自由向量，有无穷解')

    coeffi_alter = []  # 存放每个 D_j
    for i in range(len(beta)):  # 求每个 D_j
        coeffi_alter.append(copy.deepcopy(coeffi_array))  # 深拷贝，否则深层嵌套拿到的不是数值
        # coeffi_alter[i][:, i] = beta  # 实现步骤 3 中的 D_j
        for j in range(len(coeffi_array)):
            coeffi_alter[i][j][i] = beta[j]

    x = []
    for i in range(len(beta)):
        x.append(det(coeffi_alter[i]) / det_value)

    return x


def line_equation(p, q):
    ''' Ax + by = C:
    return:
        [[A, B], C]
    '''
    # 判断点坐标
    if len(q) != 2 or len(q) != 2:
        raise ValueError("此函数只可以接收二维点对。")

    A = q[1] - p[1]
    B = p[0] - q[0]
    C = p[0] * q[1] - q[0] * p[1]

    return [A, B], C


def check_result(coeffi_array, beta):
    cramer_res = np.array(cramer(coeffi_array, beta))
    linalg_solve = np.linalg.solve(np.array(coeffi_array), np.array(beta))
    if np.sum(np.power(cramer_res - linalg_solve, 2)) < 1e-8:
        print('cramer_res == linalg_solve: \n', cramer_res)
    else:
        raise ValueError('自实现结果与 numpy.linalg.solve 不符，自实现不通过。')


if __name__ == '__main__':
    # 测试1：求解方程组 demo
    coeffi_array_t1 = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]
    beta_t1 = [4, 5, 0]
    cramer(coeffi_array_t1, beta_t1)

    # 测试2：测试求两直线交点
    p1 = [1, 2]
    q1 = [2, 3]
    coefA = line_equation(p1, q1)  # p1, q1 两点所在直线
    p2 = [9, 2]
    q2 = [2, 3]
    coefB = line_equation(p2, q2)  # p2, q2 两点所在直线
    # 两直线交点
    coeffi_array = [coefA[0], coefB[0]]
    beta = [coefA[1], coefB[1]]
    cramer(coeffi_array, beta)

    # 测试3：求解方程组测试用例
    coeffi_array_t1 = [[1, 2], [3, 4]]
    beta_t1 = [4, 5]
    check_result(coeffi_array_t1, beta_t1)

    coeffi_array_t2 = [[1, 2, 3], [3, 4, 6], [4, 5, 9]]
    beta_t2 = [1, 7, 9]
    check_result(coeffi_array_t2, beta_t2)

    coeffi_array_t3 = [[1, 2, 3, 9], [3, -1, 4, 6],
                       [4, -2.5, 5, 9], [1, 2, 3, 0]]
    beta_t3 = [-3, 1, 7, 9]
    check_result(coeffi_array_t3, beta_t3)

cramer_res == linalg_solve: 
 [-3.   3.5]
cramer_res == linalg_solve: 
 [ 5.         -1.         -0.66666667]
cramer_res == linalg_solve: 
 [-27.57142857 -11.71428571  20.          -1.33333333]


In [53]:
coeffi_array_t1 = [[1, 2], [3, 4]]
beta_t1 = [4, 5]
cramer(coeffi_array_t1, beta_t1)

[-3.0, 3.5]

## 2. 高斯消元法

缺点：
1. 中间步骤频繁使用除法，导致精度损失。
    解决方案：
    - 缓解：在使主元最小，可缓解一定的损失（当前行减去主元行乘以系数，主元数字越小，系数越大）；
    - 消除：使用分数/延迟除法。
2. 需要操作原矩阵，解决方案：深拷贝一份矩阵

### 2.1 最小主元版

In [27]:
def gauss(coeffi_array_1, beta_1):
    eps = 1e-6

    # 下面操作直接在矩阵里操作，所以需要 copy，否则影响原矩阵
    coeffi_array = copy.deepcopy(coeffi_array_1)
    beta = copy.deepcopy(beta_1)

    n = len(coeffi_array)
    for index in range(n):
        # 选取当前列中最小的元素做主元
        minBaseNum = index
        for row in range(index + 1, n):
            if coeffi_array[row][index] < coeffi_array[minBaseNum][index]:
                minBaseNum = row

        if abs(coeffi_array[minBaseNum][index]) < eps:
            raise ValueError("包含自由元，无穷解")

        # 主元行交换到当前范围的最前行
        if index != minBaseNum:
            coeffi_array[index], coeffi_array[minBaseNum] = coeffi_array[minBaseNum], coeffi_array[index]
            beta[index], beta[minBaseNum] = beta[minBaseNum], beta[index]

        # 转系数矩阵为对角矩阵
        for row in range(n):
            if row == index:
                continue  # 基准行不用计算
            if coeffi_array[row][index] == 0:
                continue  # 当前行当前元素为 0 时不用计算
            rate = coeffi_array[row][index] / coeffi_array[index][index]
            for k in range(index, n):
                coeffi_array[row][k] -= coeffi_array[index][k] * rate
            beta[row] -= beta[index] * rate

    # 增广矩阵转换为简化行阶梯形式（即系数矩阵单位化），由于方程求解只需要右边部分，所以只对右边操作
    for k in range(n):
        beta[k] /= coeffi_array[k][k]
    return beta

gauss_res == linalg_solve: 
	 [-3.   3.5]
gauss_res == linalg_solve: 
	 [ 5.         -1.         -0.66666667]
gauss_res == linalg_solve: 
	 [-27.57142857 -11.71428571  20.          -1.33333333]


In [ ]:
def check_result(coeffi_array, beta):
    gauss_res = np.array(gauss(coeffi_array, beta))
    linalg_solve = np.linalg.solve(np.array(coeffi_array), np.array(beta))
    if np.sum(np.power(gauss_res - linalg_solve, 2)) < 1e-8:
        print('gauss_res == linalg_solve: \n\t', gauss_res)
    else:
        print(f'expect res: {linalg_solve}, got: {gauss_res}')
        raise ValueError('自实现结果与 numpy.linalg.solve 不符，自实现不通过。')


# 测试3：求解方程组测试用例
coeffi_array_t1 = [[1, 2], [3, 4]]
beta_t1 = [4, 5]
check_result(coeffi_array_t1, beta_t1)

coeffi_array_t2 = [[1, 2, 3], [3, 4, 6], [4, 5, 9]]
beta_t2 = [1, 7, 9]
check_result(coeffi_array_t2, beta_t2)

coeffi_array_t3 = [[1, 2, 3, 9], [3, -1, 4, 6],
                   [4, -2.5, 5, 9], [1, 2, 3, 0]]
beta_t3 = [-3, 1, 7, 9]
check_result(coeffi_array_t3, beta_t3)


### 2.2 分数版本

无精度损失，但需要依赖库。

In [28]:
from fractions import Fraction


def gauss_frac(coeffi_array_1, beta_1):
    eps = 1e-6

    # 下面操作直接在矩阵里操作，所以需要 copy，否则影响原矩阵
    coeffi_array = copy.deepcopy(coeffi_array_1)
    beta = copy.deepcopy(beta_1)

    n = len(coeffi_array)
    for index in range(n):
        # 选取不为 0 的元素做主元
        for row in range(index, n):
            if coeffi_array[row][index] > eps:
                break
        baseNum = row

        if abs(coeffi_array[baseNum][index]) < eps:
            raise ValueError("包含自由元，无穷解")

        # 主元行交换到当前范围的最前行
        if index != baseNum:
            coeffi_array[index], coeffi_array[baseNum] = coeffi_array[baseNum], coeffi_array[index]
            beta[index], beta[baseNum] = beta[baseNum], beta[index]

        # 转系数矩阵为对角矩阵
        for row in range(n):
            if row == index:
                continue  # 基准行不用计算
            if coeffi_array[row][index] == 0:
                continue  # 当前行当前元素为 0 时不用计算
            # 系数使用分数，其他元素计算使用分数后也会转型为分数
            rate = Fraction(coeffi_array[row][index],
                            coeffi_array[index][index])
            for k in range(index, n):
                coeffi_array[row][k] -= coeffi_array[index][k] * rate
            beta[row] -= beta[index] * rate

    # 增广矩阵转换为简化行阶梯形式（即系数矩阵单位化），由于方程求解只需要右边部分，所以只对右边操作
    for k in range(n):
        beta[k] = float(beta[k] / coeffi_array[k][k])
    return beta


coeffi_array_t1 = [[0, 7, 3], [0, 0, 6], [4, 5, 9]]
beta_t1 = [4, 5, 4]
gauss_frac(coeffi_array_t1, beta_t1)


[-1.1428571428571428, 0.21428571428571427, 0.8333333333333334]